# Prepare Exercice 1

In [ ]:
import pandas as pd
from Fast_connectCloud import connector
from GoogleDrivePy.google_drive import connect_drive
from GoogleDrivePy.google_platform import connect_cloud_platform
import plotly.express as px
import numpy as np
import plotly.express as px
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

In [ ]:
gs = connector.open_connection(online_connection=False,
                               path_credential='/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/')

service_gd = gs.connect_remote(engine='GS')
service_gcp = gs.connect_remote(engine='GCP')

gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])

project = 'valid-pagoda-132423'
gcp = connect_cloud_platform.connect_console(project=project,
                                             service_account=service_gcp['GoogleCloudP'])

In [ ]:
project = 'valid-pagoda-132423'
query = (
    "SELECT Customer_Name, Country_name, IncomeGroup, WB, Languages, "
    "English, French, Partnership, "
    "Inbound_Region, industry, Fixed_vodafone, "
    " Cloud_Hosting_vodafone, IoT_vodafone, "
    "Unified_Comms_vodafone, Mobility_vodafone, Mobility_potential "
    "FROM Business.Vodafone "
    #"WHERE Mobility_vodafone > 0 "
    
)

df_iden = (gcp
            .upload_data_from_bigquery(query=query,
                                       location='US')
            #.set_index(index)
            #.assign(
            #    Whitespace=lambda x: x['Mobility_potential'] - x['Mobility_vodafone'])
            )

include_all = True

(df_iden
 .loc[lambda x: (x['Mobility_potential'] <= x['Mobility_vodafone']), 
      ['Mobility_potential'] 
      #& 
     ]  
 )= df_iden['Mobility_vodafone'] 


In [ ]:
df_iden = df_iden.replace({'industry': {"Consumer Goods": "Indu_A",
                                   'Energy & Utilities': "Indu_B",
                                   'Financial Services': "Indu_C",
                                   'Insurance': "Indu_D",
                                   'Manufacturing': "Indu_E",
                                   'Professional Services': "Indu_F",
                                   'Technology': "Indu_G",
                                   'Transportation & Logistics': "Indu_H",
                                   'Travel & Tourism': "Indu_I",
                                   'Automotive': "Indu_J",
                                   'Construction': "Indu_K",
                                   'Healthcare': "Indu_L",
                                   'Media & Entertainment': "Indu_M",
                                   'Government & Public Sector': "Indu_N",
                                   'Conglomerate': "Indu_O",
                                   'IPS': "Indu_P",
                                   'Agriculture': "Indu_Q",
                                   'PRIOR-VGE': "Indu_R"
                                   },
                           'Partnership':{
                               "Non_Vodafone":'Third',
                               "Partner":'Second',
                               "Vodafone":'First',
                           }
                           })
df_final = df_iden.rename(columns={"Customer_Name": "ID",
                                    "Country_name": "Country_name",
                                    "WB": "country_ref",
                                    "Inbound_Region": "Region",
                                    "str": "industry",
                                    'Partnership': "Relationship",
                                    'Fixed_vodafone': "A_reference",
                                    'Cloud_Hosting_vodafone': "B_reference",
                                    'IoT_vodafone': "C_reference",
                                    'Unified_Comms_vodafone': "D_reference",
                                    'Mobility_vodafone': "TARGET_reference",
                                    'Mobility_potential': "TARGET_potential",
                                    })
#df_final.iloc[:, 10:] = df_final.iloc[:, 10:].transform([np.sqrt])/1000
df_final.head()

In [ ]:
df_final.to_csv(
	'dataPandasClass_UseCase.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

### Then upload to GCS
bucket_name = 'machine_learning_teaching'
destination_blob_name = 'Business_data'
source_file_name = 'dataPandasClass_UseCase.gz'
gcp.upload_blob(bucket_name, destination_blob_name, source_file_name)